# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 53 new papers today
          13 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/12 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.05520


extracting tarball to tmp_2303.05520...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.05520/h4evo.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:361: LatexWarning: Latex injecting: 'app_snr' from 'tmp_2303.05520/app_snr.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

extracting tarball to tmp_2303.05705...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.05717


extracting tarball to tmp_2303.05717...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.05741


extracting tarball to tmp_2303.05741... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.05635-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.05635) | **Assessing the spin-orbit obliquity of low-mass planets in the breaking  the chain formation model: A story of misalignment**  |
|| Leandro Esteves, et al. -- incl., <mark>Bertram Bitsch</mark> |
|*Appeared on*| *2023-03-13*|
|*Comments*| *11 pages, 9 figures, 2 tables. Accepted for publication in MNRAS*|
|**Abstract**| The spin-orbit obliquity of a planetary system constraints its formation history. A large obliquity may either indicate a primordial misalignment between the star and its gaseous disk or reflect the effect of different mechanisms tilting planetary systems after formation. Observations and statistical analysis suggest that system of planets with sizes between 1 and 4 R$_{\oplus}$ have a wide range of obliquities ($\sim0-30^{\circ}$), and that single- and multi-planet transiting have statistically indistinguishable obliquity distributions. Here, we revisit the ``breaking the chains'' formation model with focus in understanding the origin of spin-orbit obliquities. This model suggests that super-Earths and mini-Neptunes migrate close to their host stars via planet-disk gravitational interactions, forming chain of planets locked in mean-motion resonances. After gas-disk dispersal, about 90-99\% of these planetary systems experience dynamical instabilities, which spread the systems out. Using synthetic transit observations, we show that if planets are born in disks where the disk angular momentum is virtually aligned with the star's rotation spin, their final obliquity distributions peak at about $\sim$5 degrees or less, and the obliquity distributions of single and multi-planet transiting systems are statistically distinct. By treating the star-disk alignment as a free-parameter, we show that the obliquity distributions of single and multi-planet transiting systems only become statistically indistinguishable if planets are assumed to form in primordially misaligned natal disks with a ``tilt'' distribution peaking at $\gtrsim$10-20 deg. We discuss the origin of these misalignments in the context of star formation and potential implications of this scenario for formation models. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.05527-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.05527) | **The dark side of FIRE: predicting the population of dark matter  subhaloes around Milky Way-mass galaxies**  |
|| Megan Barry, et al. -- incl., <mark>Robyn Sanderson</mark> |
|*Appeared on*| *2023-03-13*|
|*Comments*| *13 pages, submitted to MNRAS*|
|**Abstract**| A variety of observational campaigns seek to test dark-matter models by measuring dark-matter subhaloes at low masses. Despite their predicted lack of stars, these subhaloes may be detectable through gravitational lensing or via their gravitational perturbations on stellar streams. To set measurable expectations for subhalo populations within LambdaCDM, we examine 11 Milky Way (MW)-mass haloes from the FIRE-2 baryonic simulations, quantifying the counts and orbital fluxes for subhaloes with properties relevant to stellar stream interactions: masses down to 10^6 Msun, distances < 50 kpc of the galactic center, across z = 0 - 1 (lookback time 0 - 8 Gyr). We provide fits to our results and their dependence on subhalo mass, distance, and lookback time, for use in (semi)analytic models. A typical MW-mass halo contains ~16 subhaloes >10^7 Msun (~1 subhalo >10^8 Msun) within 50 kpc at z = 0. We compare our results with dark-matter-only versions of the same simulations: because they lack a central galaxy potential, they overpredict subhalo counts by 2-10x, more so at smaller distances. Subhalo counts around a given MW-mass galaxy declined over time, being ~10x higher at z = 1 than at z = 0. Subhaloes have nearly isotropic orbital velocity distributions at z = 0. Across our simulations, we also identified 4 analogs of Large Magellanic Cloud satellite passages; these analogs enhance subhalo counts by 1.4-2.7 times, significantly increasing the expected subhalo population around the MW today. Our results imply an interaction rate of ~5 per Gyr for a stream like GD-1, sufficient to make subhalo-stream interactions a promising method of measuring dark subhaloes. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.05559-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.05559) | **Improved companion mass limits for Sirius A with thermal infrared  coronagraphy using a vector-apodizing phase plate and time-domain  starlight-subtraction techniques**  |
|| <mark>Joseph D. Long</mark>, et al. |
|*Appeared on*| *2023-03-13*|
|*Comments*| *19 pages, 22 figures, accepted to AJ*|
|**Abstract**| We use observations with the infrared-optimized MagAO system and Clio camera in 3.9 $\mu$m light to place stringent mass constraints on possible undetected companions to Sirius A. We suppress the light from Sirius A by imaging it through a grating vector-apodizing phase plate coronagraph with 180-degree dark region (gvAPP-180). To remove residual starlight in post-processing, we apply a time-domain principal-components-analysis-based algorithm we call PCA-Temporal (PCAT), which uses eigen-time-series rather than eigen-images to subtract starlight. By casting the problem in terms of eigen-time-series, we reduce the computational cost of post-processing the data, enabling the use of the fully sampled dataset for improved contrast at small separations. We also discuss the impact of retaining fine temporal sampling of the data on final contrast limits. We achieve post-processed contrast limits of $1.5 \times 10^{-6}$ to $9.8 \times 10^{-6}$ outside of 0.75 arcsec which correspond to planet masses of 2.6 to 8.0 $M_J$. These are combined with values from the recent literature of high-contrast imaging observations of Sirius to synthesize an overall completeness fraction as a function of mass and separation. After synthesizing these recent studies and our results, the final completeness analysis rules out 99% of $\ge 9 \ M_J$ planets from 2.5-7 AU. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.05717-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.05717) | **The effective volume of supernovae samples and sample variance**  |
|| <mark>Zhongxu Zhai</mark>, Will J. Percival |
|*Appeared on*| *2023-03-13*|
|*Comments*| *9 pages, 4 figures, comments welcome*|
|**Abstract**| The source of the tension between local SN Ia based Hubble constant measurements and those from the CMB or BAO+BBN measurements is one of the most interesting unknowns of modern cosmology. Sample variance forms a key component of the error on the local measurements, and will dominate the error budget in the future as more supernovae are observed. Many methods have been proposed to estimate sample variance in many contexts, and we compared results from a number of them in Zhai \& Percival (2022), confirming that sample variance for the Pantheon supernovae sample does not solve the Hubble tension. We now extend this analysis to include a method based on analytically calculating correlations between the radial peculiar velocities of supernovae, comparing this technique with results from numerical simulations, which can be considered a non-linear Monte-Carlo solution that works similarly. We consider the dependence of these errors on the linear power spectrum and how non-linear velocities contribute to the error. Using this technique, and matching sample variance errors, we can define an effective volume for supernovae samples, finding that the Pantheon sample is equivalent to a top-hat sphere of radius $\sim220~h^{-1}$Mpc. We use this link between sample-variance errors to compute $\Delta H_{0}$ for idealised surveys with particular angular distributions of supernovae. For example, a half-sky survey at the Pantheon depth has the potential to suppress the sample variance of $H_{0}$ to $\sim0.1$ km s$^{-1}$Mpc$^{-1}$, a significant improvement compared with the current result. Finally, we consider the strength of large-scale velocity power spectrum required to explain the Hubble tension using sample variance, finding it requires an extreme model well beyond that allowed by other observations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.05741-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.05741) | **Gluon Condensation Signature in the GeV Gamma-Ray Spectra of Pulsars**  |
|| Ze-Cheng Zou, et al. -- incl., <mark>Cheng-Ming Li</mark>, <mark>He-Rui Zheng</mark> |
|*Appeared on*| *2023-03-13*|
|*Comments*| *12 pages (9 pages for main text), 5 figures, 1 table, accepted by PRD at this https URL*|
|**Abstract**| The accumulation of gluons inside nucleons, i.e., the gluon condensation, may lead to a characteristic broken power-law gamma-ray spectrum in high-energy nucleon collisions. Here we show that the observed spectra of at least 25 sources in the second Fermi Large Area Telescope Catalog of Gamma-ray Pulsars can be well fitted by such a broken power-law function that has only four free parameters. It strongly indicates that the gamma-ray emission from these pulsars is of hadronic origin, but with gluon condensation inside hadrons. It is well known that the quark-gluon distribution in a free nucleon is different from that in a bound nucleon. This work exposes the nuclear $A$-dependence of the gluon condensation effect, where $A$ refers to the baryon number. Our study reveals the gluon condensation under the condition of $A\to\infty$, which may open a new window for eavesdropping on the structure of compact stars on the sub-nuclear level. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.05520-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.05520) | **Evolution in the orbital structure of quiescent galaxies from MAGPI,  LEGA-C and SAMI surveys: direct evidence for merger-driven growth over the  last 7 Gy**  |
|| Francesco D'Eugenio, et al. -- incl., <mark>Anna de Graaff</mark> |
|*Appeared on*| *2023-03-13*|
|*Comments*| *19 pages, 9 figures Accepted for publication in MNRAS*|
|**Abstract**| We present the first study of spatially integrated higher-order stellar kinematics over cosmic time. We use deep rest-frame optical spectroscopy of quiescent galaxies at redshifts z=0.05, 0.3 and 0.8 from the SAMI, MAGPI and LEGA-C surveys to measure the excess kurtosis $h_4$ of the stellar velocity distribution, the latter parametrised as a Gauss-Hermite series. Conservatively using a redshift-independent cut in stellar mass ($M_\star = 10^{11}\,{\rm M}_\odot$), and matching the stellar-mass distributions of our samples, we find 7 $\sigma$ evidence of $h_4$ increasing with cosmic time, from a median value of 0.019$\pm$0.002 at z=0.8 to 0.059$\pm$0.004 at z=0.06. Alternatively, we use a physically motivated sample selection, based on the mass distribution of the progenitors of local quiescent galaxies as inferred from numerical simulations; in this case, we find 10 $\sigma$ evidence. This evolution suggests that, over the last 7 Gyr, there has been a gradual decrease in the rotation-to-dispersion ratio and an increase in the radial anisotropy of the stellar velocity distribution, qualitatively consistent with accretion of gas-poor satellites. These findings demonstrate that massive galaxies continue to accrete mass and increase their dispersion support after becoming quiescent. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure f.das.h4bias.c</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.05548-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.05548) | **Light Curves and Colors of the Ejecta from Dimorphos after the DART  Impact**  |
|| Ariel Graykowski, et al. -- incl., <mark>Barton A. Smith</mark> |
|*Appeared on*| *2023-03-13*|
|*Comments*| *Accepted by Nature*|
|**Abstract**| On 26 September 2022 the Double Asteroid Redirection Test (DART) spacecraft impacted Dimorphos, a satellite of the asteroid 65803 Didymos. Because it is a binary system, it is possible to determine how much the orbit of the satellite changed, as part of a test of what is necessary to deflect an asteroid that might threaten Earth with an impact. In nominal cases, pre-impact predictions of the orbital period reduction ranged from ~8.8 - 17.2 minutes. Here we report optical observations of Dimorphos before, during and after the impact, from a network of citizen science telescopes across the world. We find a maximum brightening of 2.29 $\pm$ 0.14 mag upon impact. Didymos fades back to its pre-impact brightness over the course of 23.7 $\pm$ 0.7 days. We estimate lower limits on the mass contained in the ejecta, which was 0.3 - 0.5% Dimorphos' mass depending on the dust size. We also observe a reddening of the ejecta upon impact. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.05645-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.05645) | **Classifying Protoplanetary disks Infrared Spectrum and Analysis by  c-C$_3$H$_2$ C$_5$H$_5$ C$_9$H$_7$ C$_{12}$H$_8$ C$_{23}$H$_{12}$ and  C$_{53}$H$_{18}$ to be Capable Template for Biological Molecule**  |
|| Norio Ota, <mark>Aigen Li</mark> |
|*Appeared on*| *2023-03-13*|
|*Comments*| *10 pages, 20 figures, 1 table, 27 references*|
|**Abstract**| Protoplanetary disk around a just born young star contains a lot of cosmic dust. especially polycyclic-aromatic-hydrocarbon (PAH), which would become basic component to create biological organics. This study classified many astronomically observed infrared spectra of protoplanetary disks to three typical spectra. Type-A show well known astronomical bands of 6.2, 7.8, 8.6 and 11.3 micrometer. Whereas Type-B included unknown complex bands. Type-(A+B) was their mixed type. We tried to find specific molecule by Density Functional Theory (DFT) calculation. We found that Type-A could be explained by large PAH molecules of (C$_{23}$H$_{12}$) and (C$_{53}$H$_{18}$), which are hexagon-pentagon combined molecular structure. Background molecule of Type-B was smaller ones of (c-C$_3$H$_2$), (C$_5$H$_5$), (C$_9$H$_7$) and (C$_{12}$H$_8$). Type-(A+B) was reproduced well by mixing those molecules of A and B. Astronomical detailed observation shows that central star of Type-A has larger mass and higher temperature than that of Type-B. This suggests that at very early stage of our solar system, our protoplanetary disk had been made up by Type-B molecules. It was interesting that (C$_5$H$_5$) and (C$_9$H$_7$) of Type-B molecules has similar molecular structure with biological nucleic-acid on our earth. Type-B molecules was supposed to become the template for synthesizing biological organics and finally for creating our life. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.05635.md
    + _build/html/tmp_2303.05635/./figures/spin_orbit_obliquity.png
    + _build/html/tmp_2303.05635/./figures/compare_distributions_CDF_model3.png
    + _build/html/tmp_2303.05635/./figures/obliquity_dists_model3.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Assessing the spin-orbit obliquity of low-mass planets in the breaking the chain formation model: A story of misalignment

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.05635-b31b1b.svg)](https://arxiv.org/abs/2303.05635)<mark>Appeared on: 2023-03-13</mark> - _11 pages, 9 figures, 2 tables. Accepted for publication in MNRAS_

</div>
<div id="authors">

Leandro Esteves, et al. -- incl., <mark><mark>Bertram Bitsch</mark></mark>

</div>
<div id="abstract">

**Abstract:** The spin-orbit obliquity of a planetary system constraints its formation history. A large obliquity may either indicate a primordial misalignment between the star and its gaseous disk or reflect  the effect of different mechanisms tilting  planetary systems after formation. Observations and statistical analysis suggest that system of planets with sizes between 1 and 4 R $_{\oplus}$ have a wide range of obliquities ( $\sim0-30^{\circ}$ ), and that  single- and multi-planet transiting have statistically indistinguishable obliquity distributions. Here, we revisit the "breaking the chains" formation model with focus in understanding  the origin of spin-orbit obliquities. This model suggests that super-Earths and mini-Neptunes migrate close to their host stars via planet-disk gravitational interactions, forming chain of planets locked in mean-motion resonances. After gas-disk dispersal, about  90-99 \% of these planetary systems experience dynamical instabilities, which spread the systems out.  Using synthetic transit observations, we show that if planets are born in disks where the disk angular momentum is virtually aligned with  the star's rotation spin, their final obliquity distributions peak at  about $\sim$ 5 degrees or less, and the obliquity distributions of single and multi-planet transiting systems are statistically distinct. By treating the star-disk alignment as a free-parameter, we show that the obliquity distributions of single and multi-planet transiting systems only become statistically indistinguishable if planets are assumed to form in primordially misaligned natal disks with a  "tilt" distribution peaking at $\gtrsim$ 10-20 deg. We discuss the origin of these misalignments in the context of star formation and potential implications of this scenario for formation models.

</div>

<div id="div_fig1">

<img src="tmp_2303.05635/./figures/spin_orbit_obliquity.png" alt="Fig1" width="100%"/>

**Figure 1. -** **a)** Schematic of star-disk orientations. The star is shown as a yellow sphere at the center of the coordinate system. $\hat{n}_{\textrm{*}}$ is the norm  of the star spin vector. $\theta$ is  the angle  between $\hat{n}_{\textrm{*}}$ and the z-axis. $\Lambda$ is the azimuthal angle defined by the projection of $\hat{n}_{\textrm{*}}$ onto the x-y plane and the x-axis. The protoplanetary disk angular momentum is parallel to the z-axis. The gas disk mid-plane is shown in dark-grey.
    **b)** Star-planet orientations. The orbital plane of the planet is shown in green, which is this case, for illustration purposes do not coincides with the original disk mid-planet. The planet's angular momentum is represented by the vector $\hat{n}_{\textrm{p}}$. The spin-orbit obliquity is given by $\psi$, which is defined as the angle between $\hat{n}_{\textrm{*}}$ and $\hat{n}_{\textrm{p}}$, see inset on top left of (b). We show a single planet for simplicity. In the case, of multiple planets, the obliquity is defined using the total orbital angular momentum of all transiting planets. (*fig:obliquity_scheme*)

</div>
<div id="div_fig2">

<img src="tmp_2303.05635/./figures/compare_distributions_CDF_model3.png" alt="Fig7" width="100%"/>

**Figure 7. -** **Left panel** Cumulative distributions of obliquity ($\psi$)  of  simulated planetary systems (with no cold gas giants) and  real observations. Grey lines  show the cumulative distributions of Kepler exoplanets as inferred by  ([Louden, et. al (2021)](https://ui.adsabs.harvard.edu/abs/2021AJ....161...68L)) . In black and in colourful lines, we show the obliquity distributions of our simulated planetary systems for different values of $\sigma$. ** Right panel:** Cumulative distribution functions of $\theta$ for different values of $\sigma$.  (*fig:compare*)

</div>
<div id="div_fig3">

<img src="tmp_2303.05635/./figures/obliquity_dists_model3.png" alt="Fig4" width="100%"/>

**Figure 4. -** Cumulative distributions of obliquity ($\psi$) of simulated planetary systems divided in single and multi-planet transiting systems. Dashed lines show distributions of multi-planet transiting systems. Dotted lines represent single-transit systems. Solid lines contains all systems. Lines are colour-coded as indicated by the parameter $\sigma$. These simulations do not contain cold gas giants. (*fig:obl_dist*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

195  publications in the last 7 days.
	 _build/html/2303.05635.md
	 _build/html/2303.04727.md
	 _build/html/2303.04067.md
	 _build/html/2303.03420.md
	 _build/html/2303.02816.md
	 _build/html/2303.01528.md
	 _build/html/2303.00044.md
	 _build/html/2303.00012.md
	 _build/html/2302.14137.md
	 _build/html/2302.12805.md
	 _build/html/2302.10943.md
	 _build/html/2302.10528.md
	 _build/html/2302.10008.md
	 _build/html/2302.08962.md
	 _build/html/2302.08628.md
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
print(carousel, docs, slides)
print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.
  <div class="carousel" 
       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">
    <div class="carousel-cell"> <div id="slide1" class="md_view">Content 1</div> </div>
  </div> "2303.05635.md" "slide1"
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <!-- Remove caching as much as possible -->
    <meta http-equiv="cache-control" content="no-cache" />
    <meta http-equiv="Pragma" content="no-cache" />
    <meta http-equiv="Expires" content="-1" />
    <!-- flickity bootstrap CSS -->
    <link rel="stylesheet" href="https://unpkg.com/flickity@2/dist/flickity.min.css">
    <!-- Bootstrap CSS -->
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
        integrity="sha384-EVSTQN3/azprG1Anm